In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import io
import sys
import json

import numpy as np

import astropy.units as u
from astropy import wcs
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.visualization import make_lupton_rgb
from astropy.utils.data import download_file, clear_download_cache

import matplotlib.pyplot as plt

## Query HSC database using SQL search

* Here `unagi` provides the basic function to run quick SQL search of HSC database
* If you need to run large `SQL` search (longer than ~10 mins), please use the `CAS` online search interface

### Step 1: Setup HSC-SSP online data archive 

* First, you need to setup a HSC-SSP rerun
* Here we use the wide field from the `PDR1` as an example
* `SQL` search only cares about the data release (`dr`), you can use any `rerun` you want and still have access to all 

In [2]:
from unagi import hsc
from unagi import task
from unagi import query
from unagi import config
from unagi import plotting

pdr2 = hsc.Hsc(dr='pdr2', rerun='pdr2_dud')

# Get table list from /Users/song/Dropbox/work/project/unagi/unagi/data/pdr2_dud/pdr2_dud_tables.fits


### Basic SQL search 

* You need to write your own `SQL` script and pass it to query
* The output is a `astropy.table` table. It can be saved as an output file
* By default
    - We will skip the syntax check.  If you want syntax check, please set `skip_syntax=False`
    - We will delete the search result from the database after the search is done. If you want to keep the data, please set `delete_after=False`
    - We turn off the e-mail notification. If you don't mind being bothered by a lot of e-mails, set `noemail=False`

In [3]:
sql_test = open('../unagi/test/test_query_1.sql', 'r').read()
print(sql_test)

SELECT
	object_id
      , ra
      , dec
      , i_kronflux_mag
      , i_kronflux_magsigma
      , y_kronflux_mag
      , y_kronflux_magsigma
      , i_kronflux_mag - y_kronflux_mag AS i_y
    FROM
	pdr2_dud.forced
	JOIN pdr2_dud.forced2 USING (object_id)
    WHERE
	  boxSearch(coord, 34.0, 36.0, -5.0, -4.5)
          /* is equivalent to
                 ra  BETWEEN 34.0 AND 36.0
             AND dec BETWEEN -5.0 AND -4.5
             but boxSearch() is much faster
          */
	AND i_kronflux_mag < 25.5
    LIMIT 10
;


In [4]:
sql_file = '../unagi/test/test_query_1.sql'
result_test = pdr2.sql_query(sql_file, from_file=True, verbose=True)

print(result_test)

Waiting for query to finish... [Done]
    object_id             ra         ... y_kronflux_magsigma     i_y     
----------------- ------------------ ... ------------------- ------------
37484992795778534   34.4252353277551 ...          0.69099116   -0.6347904
37484992795811564  34.42407255027225 ...          0.16868027    1.5192337
37484992795812411  34.42568076337248 ...          0.11366094 -0.017801285
37484992795777926 34.425482291797934 ...          0.16416766   0.22124672
37484992795778758  34.42466103010599 ...           2.4685438   -1.4152412
37484992795778814  34.42460507919375 ...          0.19626102   0.57837486
37484992795779204  34.42417359236723 ...       0.00078149483    0.7457905
37484992795779508  34.42358230368066 ...         0.038728382   0.93774414
37484992795805184  34.42819665647677 ...         0.036202993   0.41768646
37484992795805232  34.42844265271551 ...            0.685199   -0.3396778


### Preview a large SQL search

* You can also test certain `SQL` search using the `preview=True` option. It will return the first 10 objects.

In [5]:
res = pdr2.sql_query(sql_test, preview=True, verbose=True)

# Preview the SQL search result...
['object_id', 'ra', 'dec', 'i_kronflux_mag', 'i_kronflux_magsigma', 'y_kronflux_mag', 'y_kronflux_magsigma', 'i_y']
['37484992795778534', '34.4252353277550966', '-4.60755454931688746', '25.3793411', '0.0927133486', '26.0141315', '0.690991163', '-0.634790421']
['37484992795811564', '34.4240725502722498', '-4.58851214394431128', '25.4374199', '0.175520271', '23.9181862', '0.168680266', '1.5192337']
['37484992795812411', '34.4256807633724833', '-4.60325464193815126', '23.4733677', '0.0276754815', '23.491169', '0.113660939', '-0.0178012848']
['37484992795777926', '34.425482291797934', '-4.61860434203913428', '25.3206272', '0.0488657616', '25.0993805', '0.164167657', '0.221246719']
['37484992795778758', '34.4246610301059874', '-4.60347780104875071', '25.4203186', '0.161276862', '26.8355598', '2.46854377', '-1.41524124']
['37484992795778814', '34.4246050791937535', '-4.60248828142772481', '25.1134052', '0.0816069618', '24.5350304', '0.196261019', '0.5783748

### Demo Search

In [6]:
pdr2 = hsc.Hsc(dr='pdr2', rerun='pdr2_dud')

# Simple example query
sql_file = '../unagi/test/test_query_1.sql'
# Result will be a astropy table
pdr2.sql_query(sql_file, from_file=True, verbose=True)

# Get table list from /Users/song/Dropbox/work/project/unagi/unagi/data/pdr2_dud/pdr2_dud_tables.fits
 [Done]


object_id,ra,dec,i_kronflux_mag,i_kronflux_magsigma,y_kronflux_mag,y_kronflux_magsigma,i_y
int64,float64,float64,float32,float32,float32,float32,float32
37484992795778534,34.4252353277551,-4.6075545493168875,25.379341,0.09271335,26.014132,0.69099116,-0.6347904
37484992795811564,34.42407255027225,-4.588512143944311,25.43742,0.17552027,23.918186,0.16868027,1.5192337
37484992795812411,34.42568076337248,-4.603254641938151,23.473368,0.027675482,23.491169,0.11366094,-0.017801285
37484992795777926,34.425482291797934,-4.618604342039134,25.320627,0.04886576,25.09938,0.16416766,0.22124672
37484992795778758,34.42466103010599,-4.603477801048751,25.420319,0.16127686,26.83556,2.4685438,-1.4152412
37484992795778814,34.42460507919375,-4.602488281427725,25.113405,0.08160696,24.53503,0.19626102,0.57837486
37484992795779204,34.42417359236723,-4.596911509544124,19.76645,0.00041926242,19.02066,0.00078149483,0.7457905
37484992795779508,34.42358230368066,-4.586865701351853,23.398798,0.0225712,22.461054,0.038728382,0.93774414
37484992795805184,34.42819665647677,-4.647357689204916,23.296112,0.012580171,22.878426,0.036202993,0.41768646


### List all the tables in the rerun

In [7]:
print(pdr2.tables(save=True))

# Read from saved file /Users/song/Dropbox/work/project/unagi/unagi/data/pdr2_dud/pdr2_dud_tables.fits
['calibframe', 'calibframe_hpx11', 'calibframe_mng', 'fcr', 'fcr_mng', 'forced', 'forced2', 'forced3', 'forced4', 'forced5', 'frame', 'frame_hpx11', 'frame_mng', 'meas', 'meas2', 'meas3', 'meas4', 'mosaic', 'mosaic_hpx11', 'mosaic_mng', 'mosaicframe', 'patch_qa', 'random', 'smallcat', 'specz', 'warped', 'warped_hpx11', 'warped_mng', 'wcs', 'wcs_mng']


### Get the schema of a table

In [8]:
forced_schema = pdr2.table_schema('forced', save=True)

# Read from saved file /Users/song/Dropbox/work/project/unagi/unagi/data/pdr2_dud/pdr2_dud_forced_schema.fits


### Build a complete list of schema for all tables

In [9]:
pdr2_wide = hsc.Hsc(dr='pdr2', rerun='pdr2_wide')

pdr2_wide_schema = pdr2_wide.build_schema()

# Get table list from /Users/song/Dropbox/work/project/unagi/unagi/data/pdr2_wide/pdr2_wide_tables.fits


In [4]:
s18a_wide = hsc.Hsc(dr='dr2', rerun='s18a_wide')
s18a_wide_schema = s18a_wide.build_schema()

s18a_dud = hsc.Hsc(dr='dr2', rerun='s18a_dud')
s18a_dud_schema = s18a_dud.build_schema()

# Querying for the table list and save it to /Users/song/Dropbox/work/project/unagi/unagi/data/s18a_wide/s18a_wide_tables.fits
# Dealing with 46 catalogs, this will take a while
# Deal with table: calibframe
# Deal with table: calibframe_anaresult
# Deal with table: calibframe_hpx11
# Deal with table: calibframe_mng
# Deal with table: fcr
# Deal with table: fcr_anaresult
# Deal with table: fcr_mng
# Deal with table: forced
# Deal with table: forced2
# Deal with table: forced3
# Deal with table: forced4
# Deal with table: forced5
# Deal with table: frame
# Deal with table: frame_anaresult
# Deal with table: frame_hpx11
# Deal with table: frame_mng
# Deal with table: masks
# Deal with table: masks_obsolate
# Deal with table: meas
# Deal with table: meas2
# Deal with table: meas3
# Deal with table: meas4
# Deal with table: mosaic
# Deal with table: mosaic_anaresult
# Deal with table: mosaic_hpx11
# Deal with table: mosaic_mng
# Deal with table: mosaicframe
# Deal with table: patch_qa
# De

### Check if a coordinate is covered by HSC 

* Use the `patch_contains` function of HSC archive to find which `Tract` and `Patch` covers a coordinate.
* Unfortunately this is not fast, it takes a few seconds to search for one object.
* And also, it only tells you that the coordinate is covered by a `Patch`. That **does not mean there is actual useful imaging data there**.

In [10]:
pdr2_wide = hsc.Hsc(dr='pdr2', rerun='pdr2_wide')

coord_1 = SkyCoord(150.0913, 2.205916, frame='icrs', unit='deg')
coord_2 = SkyCoord(150.1213, 40.235916, frame='icrs', unit='deg')

# Get table list from /Users/song/Dropbox/work/project/unagi/unagi/data/pdr2_wide/pdr2_wide_tables.fits


In [11]:
task.hsc_check_coverage(coord_1, archive=pdr2_wide, verbose=True)

# Covered by 5-band
['HSC-G', 'HSC-I', 'HSC-R', 'HSC-Y', 'HSC-Z']


tract,patch,filter01
int32,bytes64,bytes64
9813,"5,4",HSC-G
9813,"5,4",HSC-I
9813,"5,4",HSC-R
9813,"5,4",HSC-Y
9813,"5,4",HSC-Z


In [12]:
task.hsc_check_coverage(coord_2, archive=pdr2_wide, verbose=True)

# Not covered


tract,patch,filter01
int32,bytes64,bytes64
